In [ ]:
import sys
sys.path.append("../..")

import os
import pickle
import random


import pandas as pd
from pandas import DataFrame

from app.components.extract_data.extract_data import getDfPerson, getDfConsumo, getDictV9001toGroupPt, getDictV9001toGroupEn
from app.components.simple_types import Nutrition, State
from app.components.search.papa_search import SearchResult, papaSingleSeach

In [3]:
datasetPath = os.getcwd() + "/../../datasets/"
datasetPicklePath = os.getcwd() + "/../../datasets/pickle/"

In [4]:
dfConsumo = getDfConsumo()
dfPerson = getDfPerson()

In [5]:
len(dfPerson["ESTRATO_POF"].unique())

575

In [6]:
class PersonInformation:
    def __init__(self, person:str):
        self.id:str = person
        self.initialDiet:State = State.getStateByPersonId(person)        

In [7]:
class DietAgent:
    def __init__(self, person:str):
        self.personInformation:PersonInformation = PersonInformation(person)
        self.optimizedDiets:list[State] = [State.getStateByPersonId(person)]

In [8]:
class DietAllyEnemy:
    def __init__(self, person:str):
        self.enemy:int = -1
        self.ally:int = -1
        self.diet:State = State.getStateByPersonId(person) 
    

In [ ]:
class HealthAgent:
    def __init__(self):
        self.personDietAllyEnemmy:dict[str, DietAllyEnemy] = {}
        self.dieatAgents:list[DietAgent] = []
        
    def add_person(self, person:str):     
        self.personDietAllyEnemmy[person] = DietAllyEnemy(person)
        self.dieatAgents.append(DietAgent(person))
        
    def assign_social(self):
        allies = range(len(self.dieatAgents))
        enemies = range(len(self.dieatAgents))
        random.shuffle( allies )
        random.shuffle( enemies )
        
        for id in range(len(self.dieatAgents)):
            self.personDietAllyEnemmy[id].ally = allies[id]
            self.personDietAllyEnemmy[id].enemy = enemies[id]
            
            

In [10]:
personID = "110000016#7#1#5"

In [11]:
healthAgent = HealthAgent()
dietAgent = DietAgent(personID)

In [12]:
healthAgent.add_person(personID)

In [13]:
#Find people 
dfPerson = getDfPerson()
stratos = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["ESTRATO_POF"].to_list()
quantity = 10
male25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
male60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
female25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]
female60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]

In [ ]:



def multiAgentSearch(people:list[str], resultFileName:str, verbose:bool=False) -> list[list[SearchResult]]:
    try:
        with open("./output/" + resultFileName, "rb") as file:
            return pickle.load(file)
    except:
        
        healthAgent = HealthAgent()
        for person in people:
            healthAgent.add_person(person)
        healthAgent.assign_social()
        
        results = []
        phases = ["corward", "hero"]
        
        for interation in range(3):
            # RUN multiAgentSearch
            for generation in range(10):
                # operator:function = None
                # if generation%2 == 0:
                #     # TODO: Fazer o operador covarde
                #     operator = cowardOperator
                # else:
                #     # TODO: Fazer o operador heroi
                #     operator = heroOperator
                
                for agentId, agent in enumerate(healthAgent.dieatAgents):
                    result = papaSingleSeach(
                                personID=person, 
                                verbose=False, 
                                unit=10, 
                                max_unit=2, 
                                max_population_set=20, 
                                max_population_selected=10, 
                                expansion_set=50, 
                                expansion_select=10, 
                                max_steps=1, 
                                fitness=Nutrition.absDifferenceNegativePenalty, 
                                crossover=0.2, 
                                nutritionFactor=0.8,
                                distanceFactor = 0.2,
                                initialPopulation=agent.optimizedDiets,
                            )
                    
                
            
            
            # Build Results
            for i, person in enumerate(people):
                results[interation].append(
                            SearchResult(
                                [person],
                                healthAgent.dieatAgents[i].personInformation.initialDiet,
                                healthAgent.personDietAllyEnemmy[i].diet,
                            ))
        
        with open("./output/" + resultFileName, "wb") as file:
            pickle.dump(results, file) 
        
        return results
            

In [ ]:
results:list[list[SearchResult]] = []
results = multiAgentSearch(male25, "male25_100_agent.pickle", True)

100